In [ ]:
pip install twint

     |████████████████████████████████| 1.3 MB 12.8 MB/s 
     |████████████████████████████████| 263 kB 53.2 MB/s 
     |████████████████████████████████| 363 kB 44.6 MB/s 
     |████████████████████████████████| 291 kB 61.7 MB/s 
     |████████████████████████████████| 142 kB 65.2 MB/s 
     |████████████████████████████████| 294 kB 49.6 MB/s 
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33930 sha256=a45199652da99730fc96f44d66912c7c1f3659ac8f06700711545a24d1bf7fe0
  Stored in directory: /root/.cache/pip/wheels/44/fc/77/99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=3ad7a234a8f7b1583320776817bb08cee452fd2b139a873cdffd26d15bb0f2cb
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for googletransx: filename=googletransx-2.4.2-py3-none-any.whl size=15968 sha256=c30f2925e40124cc35af7d12a

In [ ]:
#Scraping
import twint
import nest_asyncio
nest_asyncio.apply()

import numpy as np
import pandas as pd
import re

#Data Scraping

The data is gathered from Twitter using Twint API. Below are the code to scrap the data using Twint. 

Steps:
1. Configure the Twint API using twint.config()
2. Provide the keywords in the search parameter. Don't forget to filter the retweets.
3. Configure the language in the Lang parameter.
4. Configure the scraped limit in the Limit parameter.
5. Configure the date using Since and Until parameter.
6. Set c.Pandas to True
7. Run the search.

**Note: Sometimes Twint might got into an error!!**

The keywords used in our search are:
1. alih fungsi lahan pertanian
2. Pertanian digusur
3. lahan pertanian habis
4. konversi lahan sawah
5. sawah jadi jalan tol
6. sawah digusur
7. Keadaan lahan pertanian
8. Kondisi lahan pertanian
9. Kebijakan ketersediaan lahan
10. penyusutan lahan pertanian
11. lahan gersang
12. Ladang pertanian
13. lahan sawah berkurang
14. konversi lahan pertanian
15. alih fungsi lahan sawah


In [ ]:
# Configuration
c = twint.Config()
c.Search = "alih fungsi lahan pertanian -filter:retweets"
c.Lang = "id"
c.Limit = 1000
c.Since = "2017-01-01"
c.Until = "2021-07-15"
c.Pandas = True

# Run 
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

In [ ]:
Tweets_df = pd.read_csv('Tweets_df.csv')

## Remove https:// and Drop Duplicates

After the data has been gathered, remove the duplicates. The tweets that contains a link like "https://" and "https://t.co/" needs also to be removed, since most tweets with link are associated on news. Sentiment analysis are not taking news as the dataset.

In [ ]:
#Remove Https://
i = 0

for tweet in Tweets_df['tweet']:
    x = re.search("https?://[A-Za-z0-9./]+", tweet)
    if x:
      Tweets_df['tweet'][i] = np.nan
    i += 1
  
Tweets_df.dropna(inplace = True)

Tweets_df = Tweets_df.reset_index()
Tweets_df.drop(['index'], axis = 1, inplace = True)

In [ ]:
#Remove Https://t.co
i = 0

for tweet in Tweets_df['tweet']:
    x = re.search("https?://t.co/[A-Za-z0-9./]+", tweet)
    if x:
      Tweets_df['tweet'][i] = np.nan
    i += 1

Tweets_df.dropna(inplace = True)

Tweets_df = Tweets_df.reset_index()
Tweets_df.drop(['index'], axis = 1, inplace = True)

In [ ]:
Tweets_df = Tweets_df.drop_duplicates(subset = ['tweet'])

In [ ]:
Tweets_df.shape

(4748, 3)

## Save the Result

Here we have done scraping the tweet with "alih fungsi lahan pertanian as the keyword". Save the result in CSV format. 

In [ ]:
Tweets_df.to_csv('alih_fungsi_lahan_pertanian.csv', index = False)

Repeat the process for the other keywords. After that, merge all the results together into one dataset. 

#Data Preprocessing

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Scraped.csv to Scraped.csv


In [ ]:
dataset = pd.read_csv('Scraped.csv')

In [ ]:
dataset.head()

,conversation_id,date,tweet
0,1.41E+18,2021-06-24 8:41:03,"@Arif_kate Di gua udh ga ada sawah, digusur se..."
1,1.41E+18,2021-06-22 1:05:52,teriak² harga properti mahal. akhirnya sawah d...
2,1.41E+18,2021-06-18 8:41:56,@hidayat_387 Gunungnya udah dikeruk. Sawah uda...
3,1.41E+18,2021-06-17 20:55:28,@TaniHitam Semua sawah akan digusur dinggo mba...
4,1.41E+18,2021-06-17 19:41:33,Terpikir jg klo fenomena skrg proyek bangunan ...


In [ ]:
dataset = dataset.drop_duplicates()

In [ ]:
dataset.shape

(4748, 3)

## Tweet Cleaning

### 1. @mention

In [ ]:
i = 0
bersih = []

for tweet in dataset['tweet']:
    bersih.append(re.sub('@[A-Za-z0-9_]+','', tweet))
    i += 1

In [ ]:
dataset['tweet'] = bersih

In [ ]:
dataset['tweet'] = dataset['tweet'].str.strip()

### 2. Hashtag

In [ ]:
i = 0
bersih = []

for tweet in dataset['tweet']:
    bersih.append(re.sub("#[a-zA-Z0-9_-]*", " ", tweet))
    i += 1

dataset['tweet'] = bersih
dataset['tweet'].str.strip()

0       Di gua udh ga ada sawah, digusur sentraland se...
1       teriak² harga properti mahal. akhirnya sawah d...
2       Gunungnya udah dikeruk. Sawah udah jadi pabrik...
3       Semua sawah akan digusur dinggo mbangun dalan ...
4       Terpikir jg klo fenomena skrg proyek bangunan ...
                              ...                        
4743    Nggak punya lahan untuk tanaman, makanya rumah...
4744    memberi latihan dan memberi pengalaman secara ...
4745    Syoknya tgk pakaian cikguÂ² ni. HariÂ² boleh p...
4746    Fungsi tanggul ini mampu melindungi kawasan la...
4747    tingkatkan proses dimana desa yg membutuhkan a...
Name: tweet, Length: 4748, dtype: object

### 3. Emoji

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
i = 0
bersih = []

for tweet in dataset['tweet']:
    bersih.append(deEmojify(tweet))
    i += 1

dataset['tweet'] = bersih
dataset['tweet'].str.strip()

0       Di gua udh ga ada sawah, digusur sentraland se...
1       teriak² harga properti mahal. akhirnya sawah d...
2       Gunungnya udah dikeruk. Sawah udah jadi pabrik...
3       Semua sawah akan digusur dinggo mbangun dalan ...
4       Terpikir jg klo fenomena skrg proyek bangunan ...
                              ...                        
4743    Nggak punya lahan untuk tanaman, makanya rumah...
4744    memberi latihan dan memberi pengalaman secara ...
4745    Syoknya tgk pakaian cikguÂ² ni. HariÂ² boleh p...
4746    Fungsi tanggul ini mampu melindungi kawasan la...
4747    tingkatkan proses dimana desa yg membutuhkan a...
Name: tweet, Length: 4748, dtype: object

### 4. NonLetter

In [ ]:
i = 0
bersih = []

for tweet in dataset['tweet']:
    bersih.append(re.sub("[^a-zA-Z]", " ", tweet))
    i += 1

dataset['tweet'] = bersih
dataset['tweet'].str.strip()

0       Di gua udh ga ada sawah  digusur sentraland se...
1       teriak  harga properti mahal  akhirnya sawah d...
2       Gunungnya udah dikeruk  Sawah udah jadi pabrik...
3       Semua sawah akan digusur dinggo mbangun dalan ...
4       Terpikir jg klo fenomena skrg proyek bangunan ...
                              ...                        
4743    Nggak punya lahan untuk tanaman  makanya rumah...
4744    memberi latihan dan memberi pengalaman secara ...
4745    Syoknya tgk pakaian cikgu   ni  Hari   boleh p...
4746    Fungsi tanggul ini mampu melindungi kawasan la...
4747    tingkatkan proses dimana desa yg membutuhkan a...
Name: tweet, Length: 4748, dtype: object

### 5. Remove Duplicates

In [ ]:
dataset.drop_duplicates(subset = ['tweet'], inplace = True)

In [ ]:
dataset.shape

(4614, 3)

## Labelling Sentiment with Inset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving negatif.csv to negatif.csv
Saving positif.csv to positif.csv


In [ ]:
pos = pd.read_csv('positif.csv')
neg = pd.read_csv('negatif.csv')

In [ ]:
pos.head()

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2


In [ ]:
neg.head()

,word,weight
0,putus tali gantung,-2
1,gelebah,-2
2,gobar hati,-2
3,tersentuh (perasaan),-1
4,isak,-5


In [ ]:
neg["digusur"] = -5
neg["dibabat"] = -4

In [ ]:
p = dict()
i = 0

for w in pos['word']:
    p[w] = pos['weight'][i]
    
n = dict()
i = 0

for w in neg['word']:
    n[w] = neg['weight'][i]

In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
    score = 0
    for word in text:
        if (word in p):
            score = score + p[word]
    for word in text:
        if (word in n):
            score = score + n[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
    return score, polarity

In [ ]:
results = dataset['tweet'].str.split().apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
dataset['polarity_score'] = results[0]
dataset['sentiment'] = results[1]
print(dataset['sentiment'].value_counts())

positive    2414
negative    1722
neutral      478
Name: sentiment, dtype: int64


### Drop the Neutrals

In [ ]:
dataset.head()

,conversation_id,date,tweet,polarity_score,sentiment
0,1.41E+18,2021-06-24 8:41:03,Di gua udh ga ada sawah digusur sentraland se...,-3,negative
1,1.41E+18,2021-06-22 1:05:52,teriak harga properti mahal akhirnya sawah d...,0,neutral
2,1.41E+18,2021-06-18 8:41:56,Gunungnya udah dikeruk Sawah udah jadi pabrik...,-2,negative
3,1.41E+18,2021-06-17 20:55:28,Semua sawah akan digusur dinggo mbangun dalan ...,-2,negative
4,1.41E+18,2021-06-17 19:41:33,Terpikir jg klo fenomena skrg proyek bangunan ...,3,positive


In [ ]:
dataset["sentiment"] = dataset['sentiment'].map({"negative" : 0, "positive" : 1})

In [ ]:
dataset.dropna(inplace = True)

In [ ]:
dataset = dataset.reset_index()
dataset.drop(['index'], axis = 1, inplace = True)

In [ ]:
dataset.shape

(4136, 5)

In [ ]:
dataset.drop(['polarity_score'], axis = 1, inplace = True)

In [ ]:
dataset.head()

,conversation_id,date,tweet,sentiment
0,1.41E+18,2021-06-24 8:41:03,Di gua udh ga ada sawah digusur sentraland se...,0.0
1,1.41E+18,2021-06-18 8:41:56,Gunungnya udah dikeruk Sawah udah jadi pabrik...,0.0
2,1.41E+18,2021-06-17 20:55:28,Semua sawah akan digusur dinggo mbangun dalan ...,0.0
3,1.41E+18,2021-06-17 19:41:33,Terpikir jg klo fenomena skrg proyek bangunan ...,1.0
4,1.41E+18,2021-06-17 15:33:53,sedih rasanya tu sawah digusur buat perumahan ...,1.0


### Save The Result

In [ ]:
dataset.to_csv('Cleaned_and_labeled.csv', index = False)

#Transform Dataset

In [ ]:
cleaned = pd.read_csv('Cleaned_and_labeled.csv')

In [ ]:
target = cleaned['sentiment']

In [ ]:
target

0       0.0
1       0.0
2       0.0
3       1.0
4       1.0
       ... 
4131    1.0
4132    1.0
4133    1.0
4134    1.0
4135    0.0
Name: sentiment, Length: 4136, dtype: float64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = cleaned['tweet'].values
vectorizer = TfidfVectorizer() ## Diambil semuanya

X = vectorizer.fit_transform(corpus).toarray()
y = cleaned["sentiment"].values

In [ ]:
X.shape

(4136, 14008)

In [ ]:
from sklearn.model_selection import train_test_split

train, test, s_train, s_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [ ]:
train.shape, test.shape, s_train.shape, s_test.shape

((2895, 14008), (1241, 14008), (2895,), (1241,))

#Modelling

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

accuracies = cross_val_score(estimator = LogisticRegression(), X = train, y = s_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 76.13 %
Standard Deviation: 2.22 %


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

logreg = LogisticRegression() 
logreg.fit(train, s_train)
cm = confusion_matrix(logreg.predict(test), s_test)
print(cm)
accuracy_score(logreg.predict(test), s_test)

[[288  60]
 [244 649]]


0.7550362610797744